# PyTorch：定义自己的自动求导函数

在底层，每一个原始的自动求导运算实际上是两个在Tensor上运行的函数。其中，**forward**函数计算从输入Tensors获得的输出Tensors。而**backward**函数接收输出Tensors对于某个标量值的梯度，并且计算输入Tensors相对于该相同标量值的梯度。 

在PyTorch中，我们可以很容易地通过定义`torch.autograd.Function`的子类并实现`forward`和`backward`函数，来定义自己的自动求导运算。之后我们就可以使用这个新的自动梯度运算符了。然后，我们可以通过构造一个实例并像调用函数一样，传入包含输入数据的tensor调用它，这样来使用新的自动求导运算。

这个例子中，我们自定义一个自动求导函数来展示ReLU的非线性。并用它实现我们的两层网络

In [1]:
# 可运行代码见本文件夹中的 two_layer_net_custom_function.py
import torch

class MyReLU(torch.autograd.Function):
    """
    我们可以通过建立torch.autograd的子类来实现我们自定义的autograd函数，
    并完成张量的正向和反向传播。
    """
    @staticmethod
    def forward(ctx, x):
        """
        在正向传播中，我们接收到一个上下文对象和一个包含输入的张量；
        我们必须返回一个包含输出的张量，
        并且我们可以使用上下文对象来缓存对象，以便在反向传播中使用。
        """
        ctx.save_for_backward(x)
        return x.clamp(min=0)

    @staticmethod
    def backward(ctx, grad_output):
        """
        在反向传播中，我们接收到上下文对象和一个张量，
        其包含了相对于正向传播过程中产生的输出的损失的梯度。
        我们可以从上下文对象中检索缓存的数据，
        并且必须计算并返回与正向传播的输入相关的损失的梯度。
        """
        x, = ctx.saved_tensors
        grad_x = grad_output.clone()
        grad_x[x < 0] = 0
        return grad_x


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# N是批大小； D_in 是输入维度；
# H 是隐藏层维度； D_out 是输出维度
N, D_in, H, D_out = 64, 1000, 100, 10

# 产生输入和输出的随机张量
x = torch.randn(N, D_in, device=device)
y = torch.randn(N, D_out, device=device)

# 产生随机权重的张量
w1 = torch.randn(D_in, H, device=device, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, requires_grad=True)

learning_rate = 1e-6
for t in range(500):
    # 正向传播：使用张量上的操作来计算输出值y；
    # 我们通过调用 MyReLU.apply 函数来使用自定义的ReLU
    y_pred = MyReLU.apply(x.mm(w1)).mm(w2)

    # 计算并输出loss
    loss = (y_pred - y).pow(2).sum()
    print(t, loss.item())

    # 使用autograd计算反向传播过程。
    loss.backward()

    with torch.no_grad():
        # 用梯度下降更新权重
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad

        # 在反向传播之后手动清零梯度
        w1.grad.zero_()
        w2.grad.zero_()

0 28410422.0
1 23503384.0
2 24120684.0
3 26386672.0
4 27314008.0
5 24231912.0
6 17958576.0
7 11102356.0
8 6172408.5
9 3319347.0
10 1876437.375
11 1162245.875
12 798493.5625
13 597297.25
14 474415.65625
15 391307.59375
16 330292.875
17 282776.0
18 244352.4375
19 212551.0
20 185790.875
21 163028.109375
22 143527.078125
23 126736.125
24 112204.875
25 99586.0625
26 88587.2890625
27 78970.0
28 70542.40625
29 63140.6171875
30 56612.61328125
31 50842.5625
32 45734.25
33 41198.7578125
34 37166.75
35 33573.1328125
36 30365.576171875
37 27499.107421875
38 24933.4296875
39 22634.890625
40 20569.662109375
41 18711.353515625
42 17036.8046875
43 15526.45703125
44 14163.080078125
45 12930.119140625
46 11814.32421875
47 10802.939453125
48 9886.189453125
49 9053.7724609375
50 8297.2822265625
51 7608.72509765625
52 6981.86572265625
53 6410.78076171875
54 5890.9462890625
55 5416.35546875
56 4982.798828125
57 4586.35400390625
58 4223.77978515625
59 3891.867919921875
60 3587.977294921875
61 3309.2255859375

446 3.858437412418425e-05
447 3.801531420322135e-05
448 3.7603087548632175e-05
449 3.713938349392265e-05
450 3.67984248441644e-05
451 3.632652806118131e-05
452 3.588247636798769e-05
453 3.5508339351508766e-05
454 3.510112583171576e-05
455 3.467723945504986e-05
456 3.4289238101337105e-05
457 3.386099706403911e-05
458 3.355701846885495e-05
459 3.317090158816427e-05
460 3.277704672655091e-05
461 3.231363007216714e-05
462 3.212285810150206e-05
463 3.168695548083633e-05
464 3.150834527332336e-05
465 3.113338607363403e-05
466 3.0756706109968945e-05
467 3.0389064704650082e-05
468 3.006316183018498e-05
469 2.971780122607015e-05
470 2.951604619738646e-05
471 2.9183574952185154e-05
472 2.87830553133972e-05
473 2.8449936507968232e-05
474 2.8168773496872745e-05
475 2.780089016596321e-05
476 2.7649279218167067e-05
477 2.7509042411111295e-05
478 2.7100619263364933e-05
479 2.6996462111128494e-05
480 2.681529258552473e-05
481 2.6524121494730935e-05
482 2.618109283503145e-05
483 2.5847204597084783e-05
